In [1]:
import os
from PIL import Image
import numpy as np
from math import ceil

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense

# Loading the Data

This code loads images from the men and women's clothing datasets from the fashion MNIST. The goal will be distinguishing between these.

To map the images to a uniform size, I use Pillow's resize function with cubic interpolation. Pillow will use cubis spline interpolation to build a map of the image, and then it will resample in the desired size. This means that images of any size can be resamples at any other size (although of course you wouldn't want to take this too far). I resample at 250x200 because this seems to strike a good balance of accuracy and speed.

In [2]:
n_imgs = 3000
size = 224, 224

men_files = ["data/men/" + s for s in os.listdir("data/men")[:n_imgs//2]]
women_files = ["data/women/" + s for s in os.listdir("data/women")[:ceil(n_imgs/2)]]

imgs = []
for file_name in men_files + women_files:
    with Image.open(file_name) as img:
        img = img.resize(size, resample=Image.BICUBIC)
        imgs.append(np.array(img).reshape(-1))

imgs = MinMaxScaler().fit_transform(np.vstack(imgs))

imgs.shape

(2512, 150528)

In [3]:
y = np.array([0]*len(men_files) + [1]*len(women_files))
x_train, x_test, y_train, y_test = train_test_split(imgs, y, test_size=0.2)

# SVM

In [4]:
print("kernel\ttrain_score\ttest_score")
print("------\t-----------\t----------")
for kernel in "linear", "poly", "rbf":
    model = SVC(kernel=kernel, degree=2)
    model.fit(x_train, y_train)
    train_score = model.score(x_train, y_train)
    test_score = model.score(x_test, y_test)
    print(f"{kernel}\t{train_score:.4f}\t\t{test_score:.4f}")

kernel	train_score	test_score
------	-----------	----------
linear	1.0000		0.5825
poly	0.9457		0.6759
rbf	0.8666		0.6899


# Feature Embedding

In [5]:
vgg16 = VGG16(weights='imagenet')

2022-03-29 05:24:27.227960: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
x_train = x_train.reshape(-1, *size, 3)
x_test = x_test.reshape(-1, *size, 3)

In [7]:
embedding_model = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)
embed_train = embedding_model.predict(x_train)
embed_test = embedding_model.predict(x_test)

In [8]:
print("kernel\ttrain_score\ttest_score")
print("------\t-----------\t----------")
for kernel in "linear", "poly", "rbf":
    model = SVC(kernel=kernel, degree=2)
    model.fit(embed_train, y_train)
    train_score = model.score(embed_train, y_train)
    test_score = model.score(embed_test, y_test)
    print(f"{kernel}\t{train_score:.4f}\t\t{test_score:.4f}")

kernel	train_score	test_score
------	-----------	----------
linear	0.9811		0.7773
poly	0.7894		0.7555
rbf	0.7765		0.7515


# Transfer Learning 

In [9]:
l1 = Dense(64, activation="relu")(embedding_model.layers[-1].output)
l2 = Dense(12, activation="relu")(l1)
l3 = Dense(1, activation="sigmoid")(l2)

transfer_model = Model(inputs=embedding_model.input, outputs=l3)
transfer_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# transfer learning
for layer in embedding_model.layers:
    layer.trainable = False
for layer in l1, l2, l3:
    layer.trainable = True
transfer_model.fit(x_train, y_train, epochs=20)

Epoch 1/20
 1/63 [..............................] - ETA: 39:27 - loss: 0.7309 - accuracy: 0.5000

In [ ]:
print(f"train score: {transfer_model.evaluate(x_train, y_train)}")
print(f"test score: {transfer_model.evaluate(x_test, y_test)}")

1/1 [==============================] - 1s 1s/step - loss: 0.2903 - accuracy: 1.0000
train score: [0.29032576084136963, 1.0]
1/1 [==============================] - 0s 436ms/step - loss: 0.6953 - accuracy: 0.5000
test score: [0.6953142881393433, 0.5]


In [ ]:
# fine tuning
for layer in embedding_model.layers:
    layer.trainable = True
transfer_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 0.2903 - accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 0.2840 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 0.2645 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 0.2455 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 1s 1s/step - loss: 0.2377 - accuracy: 1.0000


In [ ]:
print(f"train score: {transfer_model.evaluate(x_train, y_train)}")
print(f"test score: {transfer_model.evaluate(x_test, y_test)}")

1/1 [==============================] - 2s 2s/step - loss: 0.2293 - accuracy: 1.0000
train score: [0.22926774621009827, 1.0]
1/1 [==============================] - 1s 505ms/step - loss: 0.8172 - accuracy: 0.5000
test score: [0.8172279000282288, 0.5]
